In [12]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_validate, ShuffleSplit, train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
import eli5
import pandas as pd
import data 
from collections import Counter
import train
plt.style.use('dark_background')
# sns.set(style="white")
NUM_PATIENTS = 12044
# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
df = data.get_data()
feat_names = [k for k in df.keys() # features to use
              if not k in ['id', 'cv_fold'] 
              and not 'iai' in k.lower()]
outcome_def = 'iai_intervention' # output

In [16]:
print('num keys', len(df.keys()), df.keys())

num keys 64 Index(['id', 'RACE', 'HISPANIC_ETHNICITY', 'ageinyrs', 'EDTriageTime_1',
       'InjuryDay_1', 'InjTmKnown_1', 'TriageTmUnkown_1', 'Certification_1',
       'ThoracicTrauma_1', 'ThoracicTender_1', 'LtCostalTender_1',
       'RtCostalTender_1', 'DecrBreathSound_1', 'AbnChestAusc_1',
       'FlankTender_1', 'PelvicTender_1', 'PelvisUnstable_1',
       'NonAbdSurgery_1', 'CurrIntubated_1', 'PharmaParalyzed_1',
       'PharmaSedated_1', 'AlcDrgSuspicion_1', 'GCSEye_1', 'GCSVerbal_1',
       'GCSMotor_1', 'GCSScore_1', 'Preverbal_1', 'AbdomenPain_1',
       'VomitWretch_1', 'ShortBreath_1', 'DistractingPain_1',
       'InjuryMechanism_1', 'RecodedMOI_1', 'AbdUltrasound_1', 'AbdCTScan_1',
       'PtHosp_1', 'AbdTrauma_1', 'SeatBeltSign_1', 'AbdDistention_1',
       'BowelSounds_1', 'AbdomenTender_1', 'AbdTenderDegree_1',
       'PeritonIrrit_1', 'RectalBlood_1', 'InitSysBP_1', 'InitSysBPRange_1',
       'InitResp_1', 'InitRespRange_1', 'HRDocumented_1', 'InitHeartRate_1',
       

In [19]:
df.AbdTenderDegree_1.unique()

array([4., 2., 1., 3.])

# predict

In [20]:
def get_pecarn_feats(feat_names):
    '''Get dummy variable names
    '''
    pecarn_feats = ['VomitWretch_1', 'RecodedMOI_1', 'GCSScore_1', 'ThoracicTender_1', 'ThoracicTrauma_1', 
              'Cosatsl_1', 'DecrBreathSound_1', 'AbdDistention_1', 'AbdomenPain_1', 'AbdTenderDegree_1',
              'AbdTrauma_1', 'SeatBeltSign_1', 'DistractingPain_1']
    # InjuryMechanism_1, hypotension?, femure fracture
    ks = set()
    for pecarn_feat in pecarn_feats:
        for feat_name in feat_names:
            if pecarn_feat in feat_name:
                ks.add(feat_name)
    ks = np.array(list(ks))
    return ks

pecarn_feats = get_pecarn_feats(feat_names)

In [21]:
pecarn_feats

array(['GCSScore_1', 'RecodedMOI_1', 'AbdTrauma_1', 'ThoracicTrauma_1',
       'AbdDistention_1', 'AbdomenPain_1', 'SeatBeltSign_1',
       'DecrBreathSound_1', 'DistractingPain_1', 'AbdTenderDegree_1',
       'VomitWretch_1', 'ThoracicTender_1'], dtype='<U17')

In [ ]:
for balancing in ['ros']:
    for model_type in tqdm(['gb', 'logistic', 'dt', 'svm', 'rf', 'mlp2']):
        for num_feats in [5, 10, 20, 30, 52]: #111]:
            # feats = feat_names[:num_feats]
            feats = pecarn_feats[:num_feats]
            out_dir = f'results/outcome={outcome_def}'
            os.makedirs(out_dir, exist_ok=True)
            out_name = f'{model_type}_{len(feats)}_{balancing}'
            train.train(df, feat_names=feats, model_type=model_type, balancing=balancing, outcome_def=outcome_def,
                        out_name=f'{out_dir}/{out_name}.pkl')